<a href="https://colab.research.google.com/github/mafaldasalomao/pavic_treinamento_ml/blob/main/PAVIC_ML_04_Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

A melhor maneira de pensar em redes neurais é como circuitos de valores reais. Mas, ao invés de valores booleanos, valores reais e, ao invés de portas lógicas como **and** ou **or**, portas binárias (dois operandos) como $*$ (multiplicação), + (adição), max, exp, etc. Além disso, também teremos **gradientes** fluindo pelo circuito, mas na direção oposta.

In [2]:
def forwardMultiplyGate(a, b):
  return a * b

De forma matemática, a gente pode considerar que essa porta implementa a seguinte função:

$$f(x,y)=x*y$$

## O Objetivo

Vamos imaginar que temos o seguinte problema:
1. Nós vamos providenciar a um circuito valores específicos como entrada (x=1, y=-3)
2. O circuito vai calcular o valor de saída (-3)
3. A questão é: *Quanto mudar a entrada para levemente **aumentar** a saída?*

No nosso caso, em que direção devemos mudar x,y para conseguir um número maior que -3? Note que, pro nosso exemplo, se x = 0.99 e y = -2.99, x$*$y = -2.96 que é maior que -3. **-2.96 é melhor (maior) que -3**, e obtivemos uma melhora de 0.04.

## Estratégia 1: Busca Aleatória

Ok. Isso não é trivial? A gente pode simplesmente gerar valores aleatórios, calcular a saída e guardar o melhor resultado.

In [3]:
x, y = 1, -3
melhor_saida = forwardMultiplyGate(x,y)
melhor_x, melhor_y = 0, 0

for k in range(0,100):
    x_try = 5*np.random.random() - 5
    y_try = 5*np.random.random() - 5
    out = forwardMultiplyGate(x_try, y_try)
    
    if out > melhor_saida:
        melhor_saida = out
        melhor_x, melhor_y = x_try, y_try

print(melhor_x, melhor_y, forwardMultiplyGate(melhor_x, melhor_y))

-4.764806379869077 -4.871587466698096 23.21217104141332


Ok, foi bem melhor. Mas, e se tivermos milhões de entradas? É claro que essa estratégia não funcionará. Vamos tentar algo mais aprimorado.

## Estratégia 2: Busca Aleatória Local

Um passo aleatorio em qualquer direçao, torcer para tomar a decisao correta
agora temos um passo pra evitar um pulo muito longo


In [4]:
x, y = 1, -3
passo = 0.01
melhor_saida = forwardMultiplyGate(x,y)
melhor_x, melhor_y = 0, 0

for k in range(0,100):
    x_try = x + passo * (2*np.random.random() - 1)
    y_try = y + passo * (2*np.random.random() - 1)  
    out = forwardMultiplyGate(x_try, y_try)
    
    if out > melhor_saida:
        melhor_saida = out
        melhor_x, melhor_y = x_try, y_try

print(melhor_x, melhor_y, forwardMultiplyGate(melhor_x, melhor_y))

0.9914489101662263 -2.99150670342498 -2.9659260608656566


Otimoooo! Demos um passinho mais controlado -2.96 é menor que -3.... mas precisamos de algo melhor, uma estratégia mais inteligente.

## Estratégia 3: Gradiente Numérico

Imagine agora que a gente pega as entradas de um circuito e puxa-as para uma direção positiva. Essa força puxando $x$ e $y$ vai nos dizer como $x$ e $y$ devem mudar para aumentar a saída. Não entendeu? Vamos explicar:

Se olharmos para as entradas, a gente pode intuitivamente ver que a força em $y$ deveria sempre ser positiva, porque tornando $y$ um pouquinho maior de $y=1$ para $y=-1$ aumenta a saída do circuito para $-1$, o que é bem maior que $-3$. Por outro lado, se a força em $x$ for negativa, tornando-o menor, como de $x=1$ para $x=0.5$, também aumenta a saída: $-0.5\times-3 = -1.5$, de novo maior que $-3$.

E como calcular essa força? Usando **derivadas**.

> *A derivada pode ser pensada como a força que a gente aplica em cada entrada para aumentar a saída*


E como exatamente a gente vai fazer isso? Em vez de olhar para o valor de saída, como fizemos anteriormente, nós vamos iterar sobre as cada entrada individualmente, aumentando-as bem devagar e vendo o que acontece com a saída. **A quantidade que a saída muda é a resposta da derivada**.

Vamos para definição matemática. A derivada em relação a $x$ pode ser definida como:

$$\frac{\partial f(x,y)}{\partial x} = \frac{f(x+h,y) - f(x,y)}{h}$$

Onde $h$ é pequeno. Nós vamos, então, calcular a saída inicial $f(x,y)$ e aumentar $x$ por um valor pequeno $h$ e calcular a nova saída $f(x+h,y)$. Então, nós subtraimos esse valores para ver a diferença e dividimos por $f(x+h,y)$ para normalizar essa mudança pelo valor (arbitrário) que nós usamos.

Em termos de código, teremos: